In [1]:
### WE will comare runs, choose a model, and deploy it to a REST API 



In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

/opt/anaconda3/lib/python3.13/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# load the dataset
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")

data 


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [4]:
### split the data into train , test and validation sets

train, test = train_test_split(data, test_size=0.25, random_state=42)


In [5]:
train_x=train.drop("quality", axis=1).values
train_y=train["quality"].values.ravel()

## test dataset
test_x=test.drop("quality", axis=1).values
test_y=test["quality"].values.ravel()

## splitting the train data into train and validation sets

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20, random_state=42)

infer_signature(train_x, train_y)

inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [6]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define the model architechture
    mean=np.mean(train_x, axis=0)
    var=np.var(train_x, axis=0)

    model=keras.Sequential(
        [
            keras.layers.Input(shape=(train_x.shape[1],)),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)   

        ]
    )
    

    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the model with lr and momentum as hyperparameters with MLflow tracking 
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                   epochs=epochs, batch_size=64)
        
        ## Evaluate the model
        evval_result=model.evaluate(valid_x, valid_y,batch_size=64)

        eval_rmse=evval_result[1]

        ## log the parameters and results 
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        signature=infer_signature(test_x, model.predict(test_x))

        ## log the model 
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}






In [7]:
def objective(params):
    # MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [8]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)
}

In [12]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():

    # conduct hyperparameter optimization using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # fetch details of the best run
    best_run=sorted(trials.results, key=lambda x: x["loss"])[0]

    # log the best parameters, loss and model 
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    signature = infer_signature(train_x, train_y)  # Correctly infer the signature
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best Parameters:", best)
    print(f"Best  eval RMSE:", best_run["loss"])

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 11s 244ms/step - loss: 34.9270 - root_mean_squared_error: 5.9099
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6707 - root_mean_squared_error: 2.3813 - val_loss: 0.9256 - val_root_mean_squared_error: 0.9621

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.9431 - root_mean_squared_error: 0.9712
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 0.7411 - root_mean_squared_error: 0.8609 - val_loss: 0.6117 - val_root_mean_squared_error: 0.7821

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5155 - root_mean_squared_error: 0.7180
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 0.5892 - root_mean_squared_error: 0.7676 - val_loss: 0.5429 - val_root_mean_squared_error: 0.7368

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4580 - root_mean_squared_error: 0.6767
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

2026/02/13 18:42:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 204ms/step - loss: 33.3109 - root_mean_squared_error: 5.7716
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2948 - root_mean_squared_error: 1.5149 - val_loss: 0.6455 - val_root_mean_squared_error: 0.8034

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4632 - root_mean_squared_error: 0.6806
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - loss: 0.5926 - root_mean_squared_error: 0.7698 - val_loss: 0.5615 - val_root_mean_squared_error: 0.7493

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6745 - root_mean_squared_error: 0.8213
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5546 - root_mean_squared_error: 0.7447 - val_loss: 0.5425 - val_root_mean_squared_error: 0.7365

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.

2026/02/13 18:42:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 177ms/step - loss: 32.5924 - root_mean_squared_error: 5.7090
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.8296 - root_mean_squared_error: 2.4145 - val_loss: 1.7827 - val_root_mean_squared_error: 1.3352

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.5357 - root_mean_squared_error: 1.2392
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 1.4360 - root_mean_squared_error: 1.1983 - val_loss: 1.3329 - val_root_mean_squared_error: 1.1545

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.0965 - root_mean_squared_error: 1.0472
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - loss: 1.1191 - root_mean_squared_error: 1.0579 - val_loss: 1.1108 - val_root_mean_squared_error: 1.0540

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 172ms/step - loss: 35.8973 - root_mean_squared_error: 5.9914
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.4608 - root_mean_squared_error: 2.1121 - val_loss: 1.3778 - val_root_mean_squared_error: 1.1738

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.6730 - root_mean_squared_error: 1.2934
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - loss: 1.1128 - root_mean_squared_error: 1.0549 - val_loss: 1.0170 - val_root_mean_squared_error: 1.0084

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9685 - root_mean_squared_error: 0.9841
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: 0.8489 - root_mean_squared_error: 0.9214 - val_loss: 0.7746 - val_root_mean_squared_error: 0.8801

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0

2026/02/13 18:43:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:27<00:00,  6.80s/trial, best loss: 0.7365460991859436]

2026/02/13 18:43:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best Parameters: {'lr': np.float64(0.06464383844313486), 'momentum': np.float64(0.615144349651961)}
Best  eval RMSE: 0.7365460991859436
